# **RAG Fusion + Rank Reciprocal Fusion:**

In [ ]:
# install necessary libaries:

!pip install sentence_transformers -qU
!pip install langchain langchain_community chromadb pypdf langchain_google_genai -qU

## **Load The PDF documents:**

In [ ]:
!pip install unstructured pypdfium2 -qU

In [ ]:
from langchain_community.document_loaders import PyPDFium2Loader

file_path_1 = "/content/Generative AI Documentation.pdf"
loader = PyPDFium2Loader(file_path_1)
documents = loader.load()

/usr/local/lib/python3.10/dist-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [ ]:
len(documents)

36

In [ ]:
from IPython.display import display, Markdown

display(Markdown(documents[0].page_content))

Generative AI dibyendubiswas1998.gmail.com
Page | 1
Generative AI
Introduction:
Generative AI is a type of artificial intelligence technology that can produce various types of content, including text, 
imagery, audio and synthetic data. The recent buzz around generative AI has been driven by the simplicity of new 
user interfaces for creating high-quality text, graphics and videos in a matter of seconds.
The technology, it should be noted, is not brand-new. Generative AI was introduced in the 1960s in chatbots. But it 
was not until 2014, with the introduction of generative adversarial networks, or GANs -- a type of machine learning 
algorithm -- that generative AI could create convincingly authentic images, videos and audio of real people.
Generative AI refers to a class of artificial intelligence (AI) systems that are designed to generate new content or 
data rather than simply analyzing or recognizing existing patterns. These systems have the ability to create new and 
original outputs, such as images, text, music, and more, based on patterns and knowledge learned from a dataset 
during the training phase.
One popular type of generative AI is Generative Adversarial Networks (GANs). GANs consist of two neural networks 
– a generator and a discriminator – that are trained together in a competitive manner. The generator generates 
synthetic data, and the discriminator evaluates whether the generated data is real or fake. Through this adversarial 
training process, the generator improves over time, creating outputs that become increasingly difficult for the 
discriminator to distinguish from real data.
Generative AI has been applied in various fields, including image synthesis, text generation, style transfer, and even 
in creating realistic deep fake videos. While generative AI has shown remarkable capabilities, it also raises ethical 
concerns, particularly regarding the potential misuse of the technology for creating deceptive or malicious content. 
Researchers and developers are actively working on both advancing the capabilities of generative AI and addressing 
its ethical implications.
Where Generative AI exists.:
● Machine Learning is the subset of Artificial Intelligence
● Deep Learning is the subset of Machine Learning
● Generative AI is the subset of Deep Learning 


## **Chunking:**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=250)

In [ ]:
chunks = text_splitter.split_documents(documents)

In [ ]:
len(chunks)

132

In [ ]:
display(Markdown(chunks[0].page_content))

Generative AI dibyendubiswas1998.gmail.com
Page | 1
Generative AI
Introduction:
Generative AI is a type of artificial intelligence technology that can produce various types of content, including text, 
imagery, audio and synthetic data. The recent buzz around generative AI has been driven by the simplicity of new 
user interfaces for creating high-quality text, graphics and videos in a matter of seconds.
The technology, it should be noted, is not brand-new. Generative AI was introduced in the 1960s in chatbots. But it 
was not until 2014, with the introduction of generative adversarial networks, or GANs -- a type of machine learning 
algorithm -- that generative AI could create convincingly authentic images, videos and audio of real people.
Generative AI refers to a class of artificial intelligence (AI) systems that are designed to generate new content or 
data rather than simply analyzing or recognizing existing patterns. These systems have the ability to create new and

## **Load Embedding Model:**

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

## **Load LLM:**

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI


GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=GEMINI_API_KEY,
    temperature=0.5,
    max_tokens=1024,
    max_length=1024,
)

## **Insert Embeddings into Vector Database:**

In [ ]:
from langchain.vectorstores import Chroma
import chromadb

In [ ]:
DB_DIR = "ChromaDB"

client_settings = chromadb.config.Settings(
    is_persistent=True,
    persist_directory=DB_DIR,
    anonymized_telemetry=False,
)

In [ ]:
vectorstore = Chroma.from_documents(chunks,
                                    embeddings,
                                    client_settings=client_settings,
                                    collection_name="genai",
                                    collection_metadata={"hnsw":"cosine"}
              )

In [ ]:
vectorstore.similarity_search(query='What is LLM?', k=5)

[Document(metadata={'page': 9, 'source': '/content/Generative AI Documentation.pdf'}, page_content='What makes LLM so Powerful:\r\nIn case of LLM, one model can be used for a whole variety of tasks like: - Text generation, Chatbot, \r\nsummarizer, translation, code generation & so on.\r\nLLMs Model Architecture:\r\nLarge Language models are based on Transformer a type of Neural Network Architecture invented by \r\nGoogle.\r\nFew milestones in Large Language Model:\r\n\uf0b7 BERT: Bidirectional Encoder Representations from Transformers (BERT) was developed by Google.\r\n\uf0b7 GPT: GPT stands for "Generative Pre-trained “Transformer”. The model was developed by OpenAI.\r\n\uf0b7 XLM: Cross-lingual Language Model Pretraining by Guillaume Lample, Alexis Conneau.\r\n\uf0b7 T5: The Text-to-Text Transformer, it was created by Google AI.\r\n\uf0b7 Megatron: Megatron is a large, powerful transformer developed by the Applied Deep Learning \r\nResearch team at NVIDIA.\r\n\uf0b7 M2M-100: Multilin

## **Create Retriever:**

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5}, search_type="mmr")
retriever.get_relevant_documents("What is LLM?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'page': 9, 'source': '/content/Generative AI Documentation.pdf'}, page_content='What makes LLM so Powerful:\r\nIn case of LLM, one model can be used for a whole variety of tasks like: - Text generation, Chatbot, \r\nsummarizer, translation, code generation & so on.\r\nLLMs Model Architecture:\r\nLarge Language models are based on Transformer a type of Neural Network Architecture invented by \r\nGoogle.\r\nFew milestones in Large Language Model:\r\n\uf0b7 BERT: Bidirectional Encoder Representations from Transformers (BERT) was developed by Google.\r\n\uf0b7 GPT: GPT stands for "Generative Pre-trained “Transformer”. The model was developed by OpenAI.\r\n\uf0b7 XLM: Cross-lingual Language Model Pretraining by Guillaume Lample, Alexis Conneau.\r\n\uf0b7 T5: The Text-to-Text Transformer, it was created by Google AI.\r\n\uf0b7 Megatron: Megatron is a large, powerful transformer developed by the Applied Deep Learning \r\nResearch team at NVIDIA.\r\n\uf0b7 M2M-100: Multilin

## **Without RAG Fusion (Normal Way):**

In [ ]:
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

### **Create Simple Prompt:**

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

### **Create Simple Chain:**

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### **Generate Response:**

In [ ]:
chain.invoke("What is LLM?")

'Large Language models'

In [ ]:
chain.invoke("Tell me something about LLM.")

'LLM (Large Language Model) is a type of neural network architecture based on Transformer. It is capable of performing various tasks such as text generation, chatbot, summarization, translation, and code generation.'

## **With RAG Fusion + Reciprocal RAG Fusion:**

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, PromptTemplate

### **Write a prompt to generate synthetic questions based on original question:**

In [ ]:
prompt = ChatPromptTemplate(input_variables=['original_query'],
                            messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[],
                            template='You are a helpful assistant that generates multiple search queries based on a single input query.')),
                            HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))])

In [ ]:
generate_queries = (
    prompt | llm | StrOutputParser() | (lambda x: x.split("\n"))
)
generate_queries

ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant that generates multiple search queries based on a single input query.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))])
| ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), temperature=0.5, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7830ab765030>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x7830ab7981f0>, default_metadata=())
| StrOutputParser()
| RunnableLambda(lambda x: x.split('\n'))

### **Create Reciprocal Rank Fusion Function:**

In [ ]:
from langchain.load import dumps, loads


def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

### **Create refusion chain for generating synthetic questions:**

In [ ]:
ragfusion_chain = generate_queries | retriever.map() | reciprocal_rank_fusion

In [ ]:
ragfusion_chain.invoke({"question": "What is LLM?"})

### **Now Create Reciprocal Rag Fusion chain:**

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

full_rag_fusion_chain = (
    {
        "context": ragfusion_chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
full_rag_fusion_chain.input_schema.schema()

In [ ]:
full_rag_fusion_chain.invoke({"question": "Tell me about Generative AI?"})